In [1]:
# Run the command if ibm_boto3 is not installed.
!pip install ibm-cos-sdk

Requirement not upgraded as not directly required: ibm-cos-sdk in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: ibm-cos-sdk-s3transfer==2.*,>=2.0.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk)
Requirement not upgraded as not directly required: ibm-cos-sdk-core==2.*,>=2.0.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk)
Requirement not upgraded as not directly required: docutils>=0.10 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk)
Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk)
Requirement not upgraded as not directly required: python-dateutil<3.0.0,>=2.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk)
Re

In [2]:
# Install the boto library.
import ibm_boto3
from ibm_botocore.client import Config

In [3]:
# The code was removed by DSX for sharing.

In [4]:
# Define endpoint information.# Define endpoint information.
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'


In [5]:
# Define the authorization endpoint.
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

In [6]:
# Create a COS resource.
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [7]:
from uuid import uuid4

bucket_uid = str(uuid4())
buckets = ['training-data-' + bucket_uid, 'training-results-' + bucket_uid]

for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

Creating bucket "training-data-b142d891-5901-4cd0-a83c-9dbe9a19c6ac"...
Creating bucket "training-results-b142d891-5901-4cd0-a83c-9dbe9a19c6ac"...


In [8]:
# Display a list of created buckets.
print(list(cos.buckets.all()))

[s3.Bucket(name='cowactivities-donotdelete-pr-ypzouqobqc7xp3'), s3.Bucket(name='myml-donotdelete-pr-zhsoop3fasxh7h'), s3.Bucket(name='mymlcatalog-datacatalog-hj9volj77'), s3.Bucket(name='training-data-b142d891-5901-4cd0-a83c-9dbe9a19c6ac'), s3.Bucket(name='training-results-b142d891-5901-4cd0-a83c-9dbe9a19c6ac')]


In [9]:
# Create a list of links.
data_links = ['https://raw.githubusercontent.com/sinny777/hukam/master/machinelearning/tf-nlc-model/data/data.csv']

In [10]:
# Upload files from the links to COS
from urllib.request import urlopen

bucket_obj = cos.Bucket(buckets[0])

for data_link in data_links:
    filename=data_link.split('/')[-1]
    print('Uploading data {}...'.format(filename))
    with urlopen(data_link) as data:
        bucket_obj.upload_fileobj(data, filename)
        print('{} is uploaded.'.format(filename))

Uploading data data.csv...
data.csv is uploaded.


In [11]:
for bucket_name in buckets:
    print(bucket_name)
    bucket_obj = cos.Bucket(bucket_name)
    for obj in bucket_obj.objects.all():
        print("  File: {}, {:4.2f}kB".format(obj.key, obj.size/1024))

training-data-b142d891-5901-4cd0-a83c-9dbe9a19c6ac
  File: data.csv, 81.51kB
training-results-b142d891-5901-4cd0-a83c-9dbe9a19c6ac


In [14]:
!pip install wget

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [15]:
import urllib3, requests, json, base64, time, os, wget

In [16]:
wml_credentials={
  "apikey": "EKdQXYsbnxwU6q87GcWNmsc4LT9YS-vpfRo3SBZrOajK",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/2c303e22de2b34062121bab41b176b1b:e7e44faf-ff8d-4183-9f37-434e2dcd6852::",
  "iam_apikey_name": "auto-generated-apikey-b04151e7-5925-4d4e-b486-c27a83dd42cf",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/2c303e22de2b34062121bab41b176b1b::serviceid:ServiceId-7d76e768-9772-4db4-b0c7-8fcfbd2bb05a",
  "instance_id": "e7e44faf-ff8d-4183-9f37-434e2dcd6852",
  "password": "a415f1a4-38a3-4c74-9815-09cfc7791339",
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "b04151e7-5925-4d4e-b486-c27a83dd42cf"
}

In [17]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [18]:
!pip install watson-machine-learning-client --upgrade

    100% |████████████████████████████████| 1.0MB 1.0MB/s eta 0:00:01
Requirement not upgraded as not directly required: tqdm in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: urllib3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: certifi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: ibm-cos-sdk in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: lomond in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-lear

In [19]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2018-07-28 13:02:26,090 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2018-07-28 13:02:39,600 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2018-07-28 13:11:16,658 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2018-07-28 13:11:27,833 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2018-07-28 13:13:38,235 - lomond - WARNING - disconnecting websocket
2018-07-28 13:13:38,236 - lomond - ERROR - error in websocket loop
Traceback (most recent call last):
  File "/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/lomond/session.py", line 344, in run
    close_timeout):
  File "/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/lomond/session.py", line 261, in _feed
    poll, ping_rate, ping_timeout, close_timeou

In [20]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [21]:
# Display the client version number.
print(client.version)

1.0.241


In [22]:
model_definition_metadata = {
            client.repository.DefinitionMetaNames.NAME: "My NLC Model definition name",
            client.repository.DefinitionMetaNames.DESCRIPTION: "My NLC Model description",
            client.repository.DefinitionMetaNames.AUTHOR_NAME: "Gurvinder Singh",
            client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "tensorflow",
            client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "1.5",
            client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
            client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.5",
            client.repository.DefinitionMetaNames.EXECUTION_COMMAND: "python3 tf-nlc-model/tensorflow_nlc_softmax.py --training_iters 200"
            }

In [37]:
!rm -rf tf-nlc-model.zip

In [38]:
filename='tf-nlc-model.zip'

if not os.path.isfile(filename):
    filename = wget.download('https://github.com/sinny777/hukam/raw/master/machinelearning/tf-nlc-model.zip')
    
print(filename)

tf-nlc-model.zip


In [39]:
definition_details = client.repository.store_definition(filename, model_definition_metadata)

In [40]:
definition_uid = client.repository.get_definition_uid(definition_details)

In [41]:
# Display the training definition uid.
print(definition_uid)

84ff932c-5976-4dd8-8e7c-95cd7d687849


In [42]:
# Configure the training metadata for the TRAINING_DATA_REFERENCE and TRAINING_RESULTS_REFERENCE.
training_configuration_metadata = {
            client.training.ConfigurationMetaNames.NAME: "My Custom NLC Model", 
            client.training.ConfigurationMetaNames.AUTHOR_NAME: "Gurvinder Singh",              
            client.training.ConfigurationMetaNames.DESCRIPTION: "My Custom NLC Mode training",
            client.training.ConfigurationMetaNames.COMPUTE_CONFIGURATION: {"name": "k80"},
            client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCE: {
                    "connection": {
                        "endpoint_url": service_endpoint,
                        "aws_access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                        "aws_secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                    },
                    "source": {
                        "bucket": buckets[0],
                    },
                    "type": "s3"
                },
            client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
                "connection": {
                    "endpoint_url": service_endpoint,
                    "aws_access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                    "aws_secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                },
                "target": {
                    "bucket": buckets[1],
                },
                "type": "s3"
            },
        }

In [43]:
training_run_details = client.training.run(definition_uid, training_configuration_metadata)

In [44]:
training_run_guid_async = client.training.get_run_uid(training_run_details)

In [52]:
# Get training run status.
status = client.training.get_status(training_run_guid_async)
print(json.dumps(status, indent=2))

{
  "current_at": "2018-07-28T13:15:34Z",
  "finished_at": "2018-07-28T13:15:33Z",
  "state": "completed",
  "running_at": "2018-07-28T13:12:33Z",
  "metrics": [],
  "submitted_at": "2018-07-28T13:11:30Z"
}


In [53]:
client.training.monitor_logs(training_run_guid_async)



########################################################

Log monitor started for training run: training-w8G0HaFig

########################################################




-----------------
Log monitor done.
-----------------




In [ ]:
# client.training.cancel(training_run_guid_async)

In [54]:
saved_model_details = client.repository.store_model(training_run_guid_async, {"name": "My NLC Car Model"})

In [55]:
# Display location of saved model.
print("Url: " + client.repository.get_model_url(saved_model_details))

Url: https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/22b2c2f4-7041-4d50-a3f1-6a4bde68c3c1


In [56]:
# Get and display model uid.
model_uid = client.repository.get_model_uid(saved_model_details)
print("Saved model uid: " + model_uid)

Saved model uid: 22b2c2f4-7041-4d50-a3f1-6a4bde68c3c1


In [57]:
client.repository.list_models()

------------------------------------  ----------------  ------------------------  --------------
GUID                                  NAME              CREATED                   FRAMEWORK
22b2c2f4-7041-4d50-a3f1-6a4bde68c3c1  My NLC Car Model  2018-07-28T13:17:35.312Z  tensorflow-1.5
------------------------------------  ----------------  ------------------------  --------------


In [58]:
# Create online deployment.
deployment_details = client.deployments.create(model_uid, "My NLC Car model deployment")



#######################################################################################

Synchronous deployment creation for uid: '22b2c2f4-7041-4d50-a3f1-6a4bde68c3c1' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='9ed7fe34-d927-4111-8b95-0e72a3bde6f8'
------------------------------------------------------------------------------------------------




In [59]:
#  Extract endpoint url and display it.
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/e7e44faf-ff8d-4183-9f37-434e2dcd6852/deployments/9ed7fe34-d927-4111-8b95-0e72a3bde6f8/online
